In [1]:
import random
import os

import neat
from neat import DistributedEvaluator, ParallelEvaluator
from tqdm.notebook import tqdm, trange
from pathlib import Path
from random import randint
from ui.python.Layout import Layout
import numpy as np

from helpers.estimation_helpers import *
import helpers.visualize as visualize

import pandas as pd

In [2]:
MAX_WORKERS = 10
SLICE_SIZE = 400
EPOCHS = 100
SCORE_COEFFICIENTS = (500, 350, 150)
layout = Layout('./../data/layouts/genetic/step1-max/layout_4_racks.json').reset_item_count().reset_path_count()
item_list = None
check_list = None
check_ids = None
df = None
best = None
str_item = None
selected_categories = [
 'bakery',
 'beverages',
 'breakfast',
 'canned goods',
 'dairy eggs',
 'deli',
 'dry goods pasta',
 'frozen',
 'household',
 'meat seafood',
 'pantry',
 'produce',
 'snacks']

In [3]:
check_config = {
    1: 15,
    2: 25,
    3: 30,
    4: 30,
    5: 35,
    6: 30,
    7: 25,
}

In [4]:
def get_order_items(order_id):
    return order_id, list(map(str, df[df['order_id'] == order_id]['product_id'].tolist()))

def create_input_for_genome(layout, i, j, level):
    def tile_enum_to_int(tile):
        if tile.type.value == 'wall':
            return 0
        if tile.type.value == 'floor':
            return 1
        if tile.type.value == 'rack':
            return 2
        if tile.type.value == 'door':
            return 3
        if tile.type.value == 'cashier':
            return 4
        return 0

    def convert_items_to_int(items):
        ids = []
        if len(items) == 0:
            return [(-1, -1), (-1, -1), (-1, -1), (-1, -1)]
        for item in items:
            if item[0] == '':
                ids.append((-1, 0))
            else:
                ids.append((int(item[0]), int(item[1])))
        return ids

    tile_info = get_tile_info(layout, i, j)
    if tile_info is None:
        return None

    left_products = convert_items_to_int(tile_info['left_products'])
    right_products = convert_items_to_int(tile_info['right_products'])

    return [
        level,
        tile_info['dist_to_cashier'],
        tile_info['dist_to_exit'],
        tile_info['orientation'][0],
        tile_info['orientation'][1],
        left_products[0][0],
        left_products[0][1],
        left_products[1][0],
        left_products[1][1],
        left_products[2][0],
        left_products[2][1],
        left_products[3][0],
        left_products[3][1],
        right_products[0][0],
        right_products[0][1],
        right_products[1][0],
        right_products[1][1],
        right_products[2][0],
        right_products[2][1],
        right_products[3][0],
        right_products[3][1],
        tile_enum_to_int(layout[i - 1][j]),
        tile_enum_to_int(layout[i + 1][j]),
        tile_enum_to_int(layout[i][j - 1]),
        tile_enum_to_int(layout[i][j + 1]),
        tile_enum_to_int(layout[i - 1][j - 1]),
        tile_enum_to_int(layout[i + 1][j + 1]),
        tile_enum_to_int(layout[i - 1][j + 1]),
        tile_enum_to_int(layout[i + 1][j - 1]),
    ]


In [5]:
def random_layout(items_list):
    layout = Layout('./../data/layout 18x25_6.json')
    layout.set_item_list(items_list)
    for row in range(layout.shape[0]):
        for col in range(layout.shape[1]):
            if layout[row][col].type.name == 'RACK':
                for lev in range(layout.get_max_rack_level()):
                    layout.set_item_to_rack(random.choice(layout.get_item_list()), (row, col), level=lev)
    return layout

In [6]:
df = pd.read_csv('./../data/datasets/ECommerce_consumer behaviour.csv')
df = df[['order_id', 'user_id', 'order_number', 'department', 'product_id', 'product_name']]
check_list = []

df = df[df['department'].isin(selected_categories)]


def get_order_items(order_id):
    order = df[df['order_id'] == order_id]
    is_in_category = order['department'].apply(lambda x: x in selected_categories)
    return order_id, order[is_in_category]['product_name'].unique().tolist()\

# Create check list
check_ids = df['order_id'].unique().tolist()
check_list = []
for check_id in tqdm(check_ids[:10000]):
    check = get_order_items(check_id)
    check_list.append(check)

def get_checks_of_specific_length(check_list, length):
    return [x for x in check_list if len(x[1]) == length]

def get_checks_of_specific_length_range(check_list, range_dict):
    # range dict: length - n_of_checks
    res = []
    for key in tqdm(range_dict.keys()):
        res += get_checks_of_specific_length(check_list, key)[:range_dict[key]]
    return res

tuned_checks = get_checks_of_specific_length_range(check_list, check_config)

# convert tuned checks item names to item ids
check_list = []
name_id_df = df[['product_name', 'product_id']].drop_duplicates()
# ids are not sequential numbers, so we need to map them to sequential numbers
name_id_df['product_id_norm'] = range(1, len(name_id_df) + 1)
for check in tuned_checks:
    check_list.append((check[0], [str(name_id_df[name_id_df['product_name'] == x]['product_id_norm'].values[0]) for x in check[1]]))

#convert layout items to item ids
str_items = name_id_df['product_id_norm'].unique().tolist()
str_items = [str(x) for x in str_items]
layout.set_item_list(str_items, reset_items=False)
for i in range(layout.shape[0]):
    for j in range(layout.shape[1]):
        if layout[i][j].type.name == 'RACK':
            items = layout[i][j].items
            for level in range(layout.get_max_rack_level()):
                item = items[level]
                new_item = name_id_df[name_id_df['product_name'] == item[0]]['product_id_norm'].values[0]
                layout.set_item_to_rack(str(new_item), (i, j), level=level)
layout = random_layout(str_items)
best = None

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

# Data preparation

idea - take a layout, generate all possible changes for every rack, evaluate them and feed it to decision tree: state-action as X, reward diff as Y

In [7]:
data = {
    "state": [],
    "action": [],
    "reward": [], 
    "info": []
}

In [8]:
curr_info, _ = thread_func(layout, check_list)
current_reward = calculate_score(curr_info, layout, check_list, SCORE_COEFFICIENTS)

In [9]:
layout.reset_item_count()
layout.reset_path_count()

In [10]:
_, layout = thread_func(layout, check_list)

In [11]:
from concurrent.futures import ProcessPoolExecutor

for i in trange(layout.shape[0]):
    for j in trange(layout.shape[1]):
        if layout[i][j].type.name == 'RACK':
            for level in range(layout.get_max_rack_level()):
                input_data = create_input_for_genome(layout, i, j, level)
                if input_data is None:
                    continue
                futures = []
                with ProcessPoolExecutor(max_workers=MAX_WORKERS) as executor:
                    for item in str_items:
                        l = layout.copy()
                        l.set_item_to_rack(item, (i, j), level=level)
                        futures.append(executor.submit(thread_func, l, check_list))
                        data['action'].append(item)
                        data['state'].append(input_data)
                    for future in futures:
                        new_info, _ = future.result()
                        new_reward = calculate_score(new_info, l, check_list, SCORE_COEFFICIENTS)
                        data['reward'].append(new_reward - current_reward)
                        data['info'].append(new_info)

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [11]:
for k in data.keys():
    print(k, len(data[k]))

state 0
action 0
reward 0
info 0


In [12]:
# save data to file
import pickle

In [ ]:
#with open('./../data/datasets/data.pkl', 'wb') as f:
#    pickle.dump(data, f)

In [39]:
# load data from file
with open('./../data/datasets/data1.pkl', 'rb') as f:
    data = pickle.load(f)

In [40]:
data_df = pd.DataFrame(data)
data_df

,state,action,reward,info
0,"[0, 18, 28, 1, 0, -1, -1, -1, -1, -1, -1, -1, ...",1,0.000000,"{'missing_items': [], 'path': 11338, 'invalid'..."
1,"[0, 18, 28, 1, 0, -1, -1, -1, -1, -1, -1, -1, ...",2,0.000000,"{'missing_items': [], 'path': 11182, 'invalid'..."
2,"[0, 18, 28, 1, 0, -1, -1, -1, -1, -1, -1, -1, ...",3,-0.264085,"{'missing_items': [], 'path': 11320, 'invalid'..."
3,"[0, 18, 28, 1, 0, -1, -1, -1, -1, -1, -1, -1, ...",4,0.000000,"{'missing_items': [], 'path': 11476, 'invalid'..."
4,"[0, 18, 28, 1, 0, -1, -1, -1, -1, -1, -1, -1, ...",5,0.000000,"{'missing_items': [], 'path': 11346, 'invalid'..."
...,...,...,...,...
55659,"[3, 19, 11, -1, 0, -1, -1, -1, -1, -1, -1, -1,...",94,0.880282,"{'missing_items': [], 'path': 11075, 'invalid'..."
55660,"[3, 19, 11, -1, 0, -1, -1, -1, -1, -1, -1, -1,...",95,0.880282,"{'missing_items': [], 'path': 11194, 'invalid'..."
55661,"[3, 19, 11, -1, 0, -1, -1, -1, -1, -1, -1, -1,...",96,0.880282,"{'missing_items': [], 'path': 11149, 'invalid'..."
55662,"[3, 19, 11, -1, 0, -1, -1, -1, -1, -1, -1, -1,...",97,0.880282,"{'missing_items': [], 'path': 11124, 'invalid'..."


In [41]:
from sklearn.preprocessing import OneHotEncoder

#split state ito separate columns
state_df = pd.DataFrame(data_df['state'].tolist(), columns=['level', 'dist_to_cashier', 'dist_to_exit', 'orientation_x', 'orientation_y', 'left_product_1_id', 'left_product_1_count', 'left_product_2_id', 'left_product_2_count', 'left_product_3_id', 'left_product_3_count', 'left_product_4_id', 'left_product_4_count', 'right_product_1_id', 'right_product_1_count', 'right_product_2_id', 'right_product_2_count', 'right_product_3_id', 'right_product_3_count', 'right_product_4_id', 'right_product_4_count', 'tile_up', 'tile_down', 'tile_left', 'tile_right', 'tile_up_left', 'tile_down_right', 'tile_up_right', 'tile_down_left'])
# join 2 dfs
data_df = data_df.drop('state', axis=1)
data_df = data_df.join(state_df)

In [42]:
data_df

,action,reward,info,level,dist_to_cashier,dist_to_exit,orientation_x,orientation_y,left_product_1_id,left_product_1_count,...,right_product_4_id,right_product_4_count,tile_up,tile_down,tile_left,tile_right,tile_up_left,tile_down_right,tile_up_right,tile_down_left
0,1,0.000000,"{'missing_items': [], 'path': 11338, 'invalid'...",0,18,28,1,0,-1,-1,...,19,10,0,1,0,2,0,1,0,2
1,2,0.000000,"{'missing_items': [], 'path': 11182, 'invalid'...",0,18,28,1,0,-1,-1,...,19,10,0,1,0,2,0,1,0,2
2,3,-0.264085,"{'missing_items': [], 'path': 11320, 'invalid'...",0,18,28,1,0,-1,-1,...,19,10,0,1,0,2,0,1,0,2
3,4,0.000000,"{'missing_items': [], 'path': 11476, 'invalid'...",0,18,28,1,0,-1,-1,...,19,10,0,1,0,2,0,1,0,2
4,5,0.000000,"{'missing_items': [], 'path': 11346, 'invalid'...",0,18,28,1,0,-1,-1,...,19,10,0,1,0,2,0,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55659,94,0.880282,"{'missing_items': [], 'path': 11075, 'invalid'...",3,19,11,-1,0,-1,-1,...,1,10,1,0,2,0,1,0,2,0
55660,95,0.880282,"{'missing_items': [], 'path': 11194, 'invalid'...",3,19,11,-1,0,-1,-1,...,1,10,1,0,2,0,1,0,2,0
55661,96,0.880282,"{'missing_items': [], 'path': 11149, 'invalid'...",3,19,11,-1,0,-1,-1,...,1,10,1,0,2,0,1,0,2,0
55662,97,0.880282,"{'missing_items': [], 'path': 11124, 'invalid'...",3,19,11,-1,0,-1,-1,...,1,10,1,0,2,0,1,0,2,0


In [43]:
oneHotEncoder = OneHotEncoder()

def encode_data(data_df, encoder):
    encoded_df = data_df.copy()
    # encode all item columns
    encoded_df = pd.concat([encoded_df, pd.DataFrame(encoder.fit_transform(encoded_df[['left_product_1_id', 'left_product_2_id', 'left_product_3_id', 'left_product_4_id', 'right_product_1_id', 'right_product_2_id', 'right_product_3_id', 'right_product_4_id', 'action']]).toarray())], axis=1)
    
    # drop original item columns
    encoded_df = encoded_df.drop(['left_product_1_id', 'left_product_2_id', 'left_product_3_id', 'left_product_4_id', 'right_product_1_id', 'right_product_2_id', 'right_product_3_id', 'right_product_4_id', 'action', 'info'], axis=1)
    
    #drop cols with item count
    encoded_df = encoded_df.drop(['left_product_1_count', 'left_product_2_count', 'left_product_3_count', 'left_product_4_count', 'right_product_1_count', 'right_product_2_count', 'right_product_3_count', 'right_product_4_count'], axis=1)
    
    return encoded_df

encoded_df = encode_data(data_df, oneHotEncoder)

In [44]:
encoded_df

,reward,level,dist_to_cashier,dist_to_exit,orientation_x,orientation_y,tile_up,tile_down,tile_left,tile_right,...,548,549,550,551,552,553,554,555,556,557
0,0.000000,0,18,28,1,0,0,1,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0,18,28,1,0,0,1,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.264085,0,18,28,1,0,0,1,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0,18,28,1,0,0,1,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0,18,28,1,0,0,1,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55659,0.880282,3,19,11,-1,0,1,0,2,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
55660,0.880282,3,19,11,-1,0,1,0,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
55661,0.880282,3,19,11,-1,0,1,0,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
55662,0.880282,3,19,11,-1,0,1,0,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [45]:
from sklearn.utils import shuffle
data_df = shuffle(encoded_df)

In [46]:
data_df

,reward,level,dist_to_cashier,dist_to_exit,orientation_x,orientation_y,tile_up,tile_down,tile_left,tile_right,...,548,549,550,551,552,553,554,555,556,557
18,-0.264085,0,18,28,1,0,0,1,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9257,0.000000,2,23,17,-1,0,1,2,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8508,0.616197,2,19,19,-1,0,1,2,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43200,0.264085,0,13,7,0,1,2,2,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4596,0.616197,2,23,19,1,0,0,1,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27542,0.264085,1,10,16,0,-1,2,2,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38351,0.616197,3,7,17,0,-1,2,2,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2979,0.264085,2,19,21,1,0,0,1,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21561,0.000000,0,15,15,0,1,2,2,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
X = np.array(data_df.drop('reward', axis=1).values.tolist())
y = np.array(data_df['reward'].values.tolist())

In [48]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
from sklearn.metrics import mean_squared_error
import os

In [50]:
def run_regression(train_x, train_y, test_x, test_y, model, model_name, max_depth):
    model.fit(train_x, train_y)
    train_score = model.score(train_x, train_y)
    test_score = model.score(test_x, test_y)
    y_pred = model.predict(test_x)
    mse = mean_squared_error(test_y, y_pred)
    
    print(f'\'{model_name}\' --- depth: {max_depth}; train score: {train_score}; test score: {test_score}; mse: {mse}')
    
    #export_graphviz(model, out_file=f'./../data/datasets/tree-{model_name}-{max_depth}.dot', feature_names=['level', 'dist_to_cashier', 'dist_to_exit', 'orientation_x', 'orientation_y', 'left_product_1_id', 'left_product_1_count', 'left_product_2_id', 'left_product_2_count', 'left_product_3_id', 'left_product_3_count', 'left_product_4_id', 'left_product_4_count', 'right_product_1_id', 'right_product_1_count', 'right_product_2_id', 'right_product_2_count', 'right_product_3_id', 'right_product_3_count', 'right_product_4_id', 'right_product_4_count', 'tile_up', 'tile_down', 'tile_left', 'tile_right', 'tile_up_left', 'tile_down_right', 'tile_up_right', 'tile_down_left', 'action'], filled=True)
    # convert dot file to png
    #os.system('dot -Tpng ./../data/datasets/tree.dot -o ./../data/datasets/tree.png')
    return model

In [51]:
for max_depth in [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]:
    model = DecisionTreeRegressor(max_depth=max_depth)
    run_regression(X_train, y_train, X_test, y_test, model, 'decision_tree', max_depth)

'decision_tree' --- depth: 5; train score: 0.5685647075975806; test score: 0.5723680205296864; mse: 0.5245897647613077
'decision_tree' --- depth: 10; train score: 0.9083101709564053; test score: 0.8989893592092956; mse: 0.12391296917601821
'decision_tree' --- depth: 15; train score: 0.9294387860990986; test score: 0.9185881147450262; mse: 0.09987055174774659
'decision_tree' --- depth: 20; train score: 0.9600456450892031; test score: 0.9456401100920103; mse: 0.06668500773633687
'decision_tree' --- depth: 25; train score: 0.9876070500454459; test score: 0.9733292861190217; mse: 0.03271781389728791
'decision_tree' --- depth: 30; train score: 0.9987743391023199; test score: 0.9819306238036706; mse: 0.02216627909810954
'decision_tree' --- depth: 35; train score: 0.9998003048633102; test score: 0.9818746779586939; mse: 0.022234909647424566
'decision_tree' --- depth: 40; train score: 0.9999558788995025; test score: 0.9817640790087664; mse: 0.022370584900704624
'decision_tree' --- depth: 45; t

In [52]:
model = DecisionTreeRegressor(max_depth=20)

In [53]:
# fit model
model = run_regression(X_train, y_train, X_test, y_test, model, 'decision_tree', 30)

'decision_tree' --- depth: 30; train score: 0.96004740466664; test score: 0.945578831998322; mse: 0.06676017952492187


In [54]:
# run model for each item in the layout

def generate_data_for_layout(layout, encoder):
    data = {
        "state": [],
        "action": [],
        "reward": [],
        "info": [],
        "coords": []
    }
    
    for i in range(layout.shape[0]):
        for j in range(layout.shape[1]):
            if layout[i][j].type.name == 'RACK':
                for level in range(layout.get_max_rack_level()):
                    input_data = create_input_for_genome(layout, i, j, level)
                    if input_data is None:
                        continue
                    for item in str_items:
                        data['action'].append(item)
                        data['state'].append(input_data)
                        data['info'].append(None)
                        data['reward'].append(None)
                        data['coords'].append((i, j, level))
    df = pd.DataFrame(data)
    state_df = pd.DataFrame(df['state'].tolist(), columns=['level', 'dist_to_cashier', 'dist_to_exit', 'orientation_x', 'orientation_y', 'left_product_1_id', 'left_product_1_count', 'left_product_2_id', 'left_product_2_count', 'left_product_3_id', 'left_product_3_count', 'left_product_4_id', 'left_product_4_count', 'right_product_1_id', 'right_product_1_count', 'right_product_2_id', 'right_product_2_count', 'right_product_3_id', 'right_product_3_count', 'right_product_4_id', 'right_product_4_count', 'tile_up', 'tile_down', 'tile_left', 'tile_right', 'tile_up_left', 'tile_down_right', 'tile_up_right', 'tile_down_left'])
    # join 2 dfs
    df = df.drop('state', axis=1)
    df = df.join(state_df)
    enc_df = encode_data(df, encoder)
    return enc_df

In [55]:
data = generate_data_for_layout(layout, oneHotEncoder)
data

,reward,coords,level,dist_to_cashier,dist_to_exit,orientation_x,orientation_y,tile_up,tile_down,tile_left,...,647,648,649,650,651,652,653,654,655,656
0,None,"(1, 2, 0)",0,18,28,1,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,None,"(1, 2, 0)",0,18,28,1,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,None,"(1, 2, 0)",0,18,28,1,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,None,"(1, 2, 0)",0,18,28,1,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,None,"(1, 2, 0)",0,18,28,1,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55659,None,"(16, 22, 3)",3,19,11,-1,0,1,0,2,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
55660,None,"(16, 22, 3)",3,19,11,-1,0,1,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
55661,None,"(16, 22, 3)",3,19,11,-1,0,1,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
55662,None,"(16, 22, 3)",3,19,11,-1,0,1,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [56]:
# fit without reward and coords
X = np.array(data.drop(['reward', 'coords'], axis=1).values.tolist())

#predict rewards
y = model.predict(X)

#add rewards to data
data['reward'] = y

ValueError: X has 670 features, but DecisionTreeRegressor is expecting 571 features as input.

In [ ]:
# for each item in layout, select the one with the highest reward
def select_best_item_for_layout(layout, data, rewards):
    new_layout = layout.copy()
    for i in trange(layout.shape[0]):
        for j in range(layout.shape[1]):
            if layout[i][j].type.name == 'RACK':
                for level in range(layout.get_max_rack_level()):
                    coords = (i, j, level)
                    coords_df = data[data['coords'] == coords]
                    decoded = oneHotEncoder.inverse_transform(coords_df[[x for x in range(len(coords_df.columns)-15)]].values.tolist())
                    decoded_df = pd.DataFrame(decoded, columns=['left_product_1_id', 'left_product_2_id', 'left_product_3_id', 'left_product_4_id', 'right_product_1_id', 'right_product_2_id', 'right_product_3_id', 'right_product_4_id', 'action'])
                    #print(decoded_df)
                    decoded_df = decoded_df.join(coords_df[['reward']])
                    decoded_df.sort_values(by='reward', ascending=False)
                    #(coords_df)
                    item = decoded_df.iloc[random.randint(0, 4)]['action']
                    #print(item, type(item))
                    new_layout.set_item_to_rack(item, (i, j), level=level)
    return new_layout

In [ ]:
layout = layout.reset_item_count().reset_path_count().copy()

In [ ]:
better_layout = layout.copy()

In [ ]:
for i  in range(5):
    better_layout = better_layout.reset_item_count().reset_path_count()
    better_layout = select_best_item_for_layout(better_layout, data, y)
    res, l = thread_func(better_layout, check_list)
    print(res)

In [ ]:
eval = thread_func(layout, check_list)
test = thread_func(better_layout, check_list)

In [ ]:
eval

In [ ]:
test

In [ ]:
dir = str(Path(os.getcwd()).parent)

In [ ]:
better_layout.display_in_window(home_dir=dir)